In [ ]:
%pip install transformers datasets peft wandb

In [1]:
import transformers
import datasets 
import peft

In [2]:
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import load_dataset, Dataset, DatasetDict

lora_config = LoraConfig(
    r=4, # number of the parameters to train.
    lora_alpha=1, # magnitude of the weight matrix
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type="SEQ_CLS" # sequence classification
)

In [3]:
from transformers import BertTokenizer, BertModel
model_checkpoint = "google-bert/bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint, use_fast=True)
foundation_model = BertModel.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
import pandas as pd

DATA_PATH = "/home/jovyan/work/ULM-25-authorship-profiling/data/"

df_train = pd.read_csv(DATA_PATH + "data_train.csv")
df_test = pd.read_csv(DATA_PATH + "data_test.csv")
df_val = pd.read_csv(DATA_PATH + "data_val.csv")

In [5]:
dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(df_train)
dataset['validation'] = Dataset.from_pandas(df_val)
dataset['test'] = Dataset.from_pandas(df_test)

In [6]:
dataset = dataset.filter(lambda example: 
                         example["text"] is not None and 
                         example["gender"] is not None and
                         example ["age"] is not None)
dataset

Filter:   0%|          | 0/620813 [00:00<?, ? examples/s]

Filter:   0%|          | 0/68980 [00:00<?, ? examples/s]

Filter:   0%|          | 0/37919 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 620812
    })
    validation: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 68980
    })
    test: Dataset({
        features: ['text', 'age', 'gender'],
        num_rows: 37919
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModel, PreTrainedModel, PretrainedConfig
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import numpy as np


def preprocess_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
    gender_labels = [
        {"male": 0, "female": 1}[label] for label in examples["gender"]
    ]
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "age_labels": age_labels,
        "gender_labels": gender_labels,
        "labels": np.stack([age_labels, gender_labels], axis=1)
    }

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['text', 'age', 'gender']
)

In [ ]:
tokenized_dataset

In [ ]:
from huggingface_hub import login
login()

In [ ]:
tokenized_dataset.push_to_hub(
    "KonradBRG/ULM-Profiling-tokenized",
    private=False,  # or True if you want it private
    commit_message="Upload tokenized ULM profiling dataset"
)

In [ ]:
class JointClassificationConfig(PretrainedConfig):
    def __init__(self, num_age_labels=None, num_gender_labels=None, loss_alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.num_age_labels = num_age_labels
        self.num_gender_labels = num_gender_labels

In [ ]:
from transformers.modeling_outputs import SequenceClassifierOutput
from torch.nn import CrossEntropyLoss

class BertForJointClassification(PreTrainedModel):
    config_class = JointClassificationConfig
    
    def __init__(self, config, model):
        super().__init__(config)
        self.num_age_labels = config.num_age_labels
        self.num_gender_labels = config.num_gender_labels

        self.bert = model
        self.age_classifier = nn.Linear(config.hidden_size, self.num_age_labels)
        self.gender_classifier = nn.Linear(config.hidden_size, self.num_gender_labels)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.age_loss_fct = CrossEntropyLoss()
        self.gender_loss_fct = CrossEntropyLoss()
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        age_labels=None,
        gender_labels=None,
        **kwargs
    ):
        # get BERT outputs
        x = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        # pool outputs to get sequence representation
        x = x.last_hidden_state[:, 0]
        x = self.dropout(x)
        # get logits from task head
        age_logits = self.age_classifier(x)
        gender_logits = self.gender_classifier(x)
        loss_age = self.age_loss_fct(age_logits, age_labels)
        loss_gender = self.gender_loss_fct(gender_logits, gender_labels)
        return SequenceClassifierOutput(loss={"loss_age": loss_age, "loss_gender": loss_gender}, 
                                        logits=torch.cat([age_logits, gender_logits], dim=1))

In [ ]:
def setup_model(dataset, model_name="bert-base-uncased", peft=True):        
    config = JointClassificationConfig(
        name_or_path=model_name,
        num_age_labels=num_age_labels,
        num_gender_labels=2,
        hidden_size=768,
        hidden_dropout_prob=0.1,
    )
    bert = BertForJointClassification(config, foundation_model)
    if peft:
        bert = get_peft_model(bert, lora_config)
    return bert, config, dataset

In [ ]:
# optional
peft_model.unload()

In [ ]:
peft_model, config, data = setup_model(tokenized_dataset, model_checkpoint)

In [ ]:
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    age_labels, gender_labels = eval_pred.label_ids

    age_logits = logits[:, :num_age_labels]
    gender_logits = logits[:, num_age_labels:]

    age_preds = np.argmax(age_logits, axis=-1)
    gender_preds = np.argmax(gender_logits, axis=-1)

    age_acc = (age_preds == age_labels).mean()
    gender_acc = (gender_preds == gender_labels).mean()
    joint_acc = np.mean((age_preds == age_labels) & (gender_preds == gender_labels))

    return {"age_acc": age_acc, "gender_acc": gender_acc, "joint_acc": joint_acc}

In [ ]:
from typing import Dict, Union, Any
from transformers import Trainer, TrainingArguments, default_data_collator

class TrainerWithCustomLoss(Trainer):
    
    def __init__(self, age_alpha: float = 0.5, scale_losses = True, **kwargs):
        super().__init__(**kwargs)
        self.age_alpha = age_alpha
        self.gender_alpha = 1 - self.age_alpha
        self._scale_losses = scale_losses
    
    def training_step(
        self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]], num_items_in_batch=None
    ) -> torch.Tensor:
        model.train()
        if hasattr(self.optimizer, "train") and callable(self.optimizer.train):
            self.optimizer.train()
        inputs = self._prepare_inputs(inputs)
        
        with self.compute_loss_context_manager():
            loss = self.compute_loss(model, inputs)
        self.accelerator.backward(loss)
        
        # Finally we need to normalize the loss for reporting
        if num_items_in_batch is None:
            return loss.detach() / self.args.gradient_accumulation_steps
        return loss.detach()

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)
        loss_age, loss_gender = outputs.loss["loss_age"], outputs.loss["loss_gender"]
        loss = self.age_alpha * loss_age + self.gender_alpha * loss_gender
        
        if return_outputs:
            return loss, outputs
        return loss

In [ ]:
del trainer

In [ ]:
from transformers import Trainer, TrainingArguments, default_data_collator

print(peft_model.print_trainable_parameters())
log_frequency = 1000

training_args = TrainingArguments(
    "bert-lora-for-author-profiling",
    per_device_train_batch_size=32,  # Set explicitly for better control
    gradient_accumulation_steps=2,   # Effective batch size = 32 * 2 = 64
    num_train_epochs=3,
    # Speed optimizations
    dataloader_num_workers=4,        # Parallel data loading
    dataloader_pin_memory=True,      # Faster GPU transfer
    bf16=True,                       # Mixed precision (if you have newer GPU)
    # fp16=True,                     # Use this instead if bf16 not supported
    # Reduce overhead
    eval_strategy="steps",           # Less frequent evaluation
    eval_steps=500,                  # Adjust based on dataset size
    logging_strategy="steps",
    logging_steps=log_frequency,
    save_strategy="no",
    learning_rate=5e-5,
    report_to="wandb"
)

trainer = TrainerWithCustomLoss(
    age_alpha=0.7,
    model=peft_model,
    args=training_args,
    train_dataset=data['train'].with_format("torch"),
    eval_dataset=data['validation'].with_format("torch"),
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.predict(data["test"])

In [ ]:
import wandb
wandb.finish()
sweep_config = {
    "name": "LoRA-For-Author-Profiling",
    "method": "bayes",
    "metric": {
        "name": "Joint Acc",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "min": 0.00005,
            "max": 0.0001
        },
        "age_alpha": {
            "min": 0.4,
            "max": 0.9
        },
        "per_device_train_batch_size": {
            "values": [16, 32, 64]
        },
        "num_train_epochs": {
            "values": [2, 3, 4]
        },
        "peft_r_value": {
            "values": [2, 4, 8]
        },
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project="ULM-Author-Profiling")

In [ ]:
def train(config=None):
    # Initialize wandb run with the config from sweep
    with wandb.init(config=config):
        peft_r_value = wandb.config.peft_r_value
        learning_rate = wandb.config.learning_rate
        age_alpha = wandb.config.age_alpha
        batch_size = wandb.config.per_device_train_batch_size
        epochs = wandb.config.num_train_epochs

        lora_config = LoraConfig(
            r=peft_r_value, # number of the parameters to train.
            lora_alpha=1, # magnitude of the weight matrix
            target_modules=["query", "value"],
            lora_dropout=0.05,
            bias="lora_only",
            task_type="SEQ_CLS" # sequence classification
        )
        peft_model, config, data = setup_model(tokenized_dataset, model_checkpoint)

        # Set up training arguments
        training_args = TrainingArguments(
            output_dir="bert-lora-for-author-profiling",
            per_device_train_batch_size=batch_size,
            gradient_accumulation_steps=2,
            num_train_epochs=epochs,
            # Speed optimizations
            dataloader_num_workers=4,    
            dataloader_pin_memory=True,
            bf16=True,
            eval_strategy="steps",         
            eval_steps=1000,                  
            logging_strategy="steps",
            logging_steps=1000,
            save_strategy="no",
            learning_rate=learning_rate,
            report_to="wandb"
        )
        
        trainer = TrainerWithCustomLoss(
            age_alpha=age_alpha,
            model=peft_model,
            args=training_args,
            train_dataset=data['train'].with_format("torch"),  # Make sure data is defined
            eval_dataset=data['validation'].with_format("torch"),
            data_collator=default_data_collator,
            compute_metrics=compute_metrics,
        )
        try:
            trainer.train()
            eval_results = trainer.evaluate()
            wandb.log(eval_results)
            print(f"Training completed! Final Joint Acc: {eval_results.get('eval_Joint Acc', 'N/A')}")
        finally:
            # Ensure cleanup happens even if training fails
            peft_model.unload()
            del trainer
            del peft_model
            torch.cuda.empty_cache()  # Clear GPU memory

In [ ]:
wandb.agent(sweep_id, train, count=10)  # Will run 10 different hyperparameter combinations

In [ ]:
trainer.push_to_hub()